<a href="https://colab.research.google.com/github/JonScottE/RobinHood_Bot/blob/main/RobinhoodBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install robin_stocks

import requests
import pandas as pd
import numpy as np
import talib
import robin_stocks as rs

rs.login(username='username_here',
         password='Password_here',
         expiresIn=86400,
         by_sms=True)

def get_data(symbol, interval='5minute'):
    data = rs.stocks.get_crypto(symbol, interval=interval)
    return data

def buy_sell_signal(df):
    close = df['Close']
    rsi = talib.RSI(close, timeperiod=14)
    signals = []
    for i in range(1, len(df)):
        if rsi[i-1] < 80 and rsi[i] >= 80:
            signals.append(1) # Buy signal
        elif rsi[i-1] > 80 and rsi[i] <= 80:
            signals.append(-1) # Sell signal
        else:
            signals.append(0)
    df['Signal'] = signals
    return df

def backtesting(df):
    signals = df['Signal']
    close = df['Close']
    position = 0
    PnL = [0]
    for i in range(1, len(df)):
        if signals[i] == 1:
            position = 100/close[i]
        elif signals[i] == -1:
            PnL.append(position*close[i]-100)
            position = 0
        else:
            PnL.append(position*close[i]-100)
    df['PnL'] = PnL
    return df

if __name__ == '__main__':
    symbol = 'AAPL'
    data = get_data(symbol)
    df = pd.DataFrame(data)
    df = df[['close_price', 'begins_at']]
    df = df.rename(columns={'close_price': 'Close', 'begins_at': 'Date'})
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.set_index('Date')
    df = df.dropna()
    df = buy_sell_signal(df)
    df = backtesting(df)
    print(df)
